In [1]:
import dill
import pickle
from pprint import pprint
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from Settings import Settings
from window_based_tagger_config import get_config
from FindFiles import find_files
from DirUtils import dir_exists
from results_procesor import ResultsProcessor, __MICRO_F1__

In [16]:
EMPTY_TAG = "Empty"
ANAPHORA = "Anaphor"

In [3]:
!python -V

Python 3.6.4 :: Anaconda, Inc.


In [4]:
!pip freeze

absl-py==0.3.0
anaconda-client==1.6.11
appnope==0.1.0
argcomplete==1.9.4
asn1crypto==0.24.0
astor==0.7.1
beautifulsoup4==4.6.0
bleach==2.1.2
boto==2.47.0
boto3==1.5.36
botocore==1.8.50
bz2file==0.98
certifi==2018.1.18
cffi==1.11.4
chardet==3.0.4
clyent==1.2.2
costcla==0.5
cryptography==2.1.4
cycler==0.10.0
cymem==1.31.2
cytoolz==0.8.2
decorator==4.2.1
dicecore==1.13
dill==0.2.8.2
docutils==0.14
entrypoints==0.2.3
ftfy==4.4.3
gast==0.2.0
gensim==0.13.4
grpcio==1.14.0
h5py==2.7.0
hdbscan==0.8.12
html5lib==1.0.1
idna==2.6
ipykernel==4.8.2
ipython==6.2.1
ipython-genutils==0.2.0
ipywidgets==7.1.2
jedi==0.11.1
Jinja2==2.10
jmespath==0.9.3
joblib==0.9.4
jsonschema==2.6.0
jupyter==1.0.0
jupyter-client==5.2.2
jupyter-console==5.2.0
jupyter-core==4.4.0
jupyterlab==0.31.8
jupyterlab-launcher==0.10.5
Keras==1.2.2
Keras-Applications==1.0.4
Keras-Preprocessing==1.0.2
Markdown==2.6.11
MarkupSafe==1.0
matplotlib==2.0.0
mistune==0.8.3
murmurhash==0.26.4
nb-anacondacloud==1.4.0
nb-conda==2.2.1
nb-conda-

In [19]:
#DATASET = "CoralBleaching"
DATASET = "SkinCancer"

settings = Settings()
root_folder = settings.data_directory + DATASET + "/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
training_pickled = settings.data_directory + DATASET + "/Thesis_Dataset/training.pl"

# PREDICTIONS FOLDERS
#anaphor_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-Anaphora_Tags-Binary-Fixed/"
anaphor_predictions_folder = root_folder + "Predictions/Bi-LSTM-4-Anaphora_Tags-Binary/"
tag_predictions_folder = root_folder + "Predictions/Bi-LSTM_fixed/"

config = get_config(training_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [20]:
anaphor_predictions_folder

'/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM-4-Anaphora_Tags-Binary/'

In [21]:
assert dir_exists(anaphor_predictions_folder)
print("Valid")

Valid


In [22]:
#!ls '/Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/Predictions/Bi-LSTM-4-Anaphora_Tags-Binary/'

In [23]:
tag_predictions_folder

'/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM_fixed/'

In [24]:
assert dir_exists(tag_predictions_folder)
print("Valid")

Valid


In [25]:
# get tagged essays
def load_tagged_essays(folder):
    files = find_files(folder, "essay.*.dill")
    # multiple runs with different hidden layer sizes?
    if len(files) > 2:        
        files = find_files(folder, "essays.*256.*.dill")
    for f in files:
        print(f)
    assert len(files) == 2, "Wrong number of tagged files:" + str(len(files))
    for f in files:
        assert "_train_" in f or "_test_" in f, "Wrong files matched"
    train_tagged_fname = [f for file in files if "_train_" in f][0]
    test_tagged_fname = list(set(files).difference([train_tagged_fname]))[0]

    # NOTE - is this throws an error, upgrade to dill 2.8.2. Version 2.6 had a bug in it
    with open(train_tagged_fname, "rb") as f:
        tagged_essays_train = dill.load(f)
    with open(test_tagged_fname, "rb") as f:
        tagged_essays_test  = dill.load(f)
    return (tagged_essays_train, tagged_essays_test)

In [26]:
ana_tagged_tr, ana_tagged_test = load_tagged_essays(anaphor_predictions_folder)
len(ana_tagged_tr), len(ana_tagged_test)

/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM-4-Anaphora_Tags-Binary/essays_test_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill
/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM-4-Anaphora_Tags-Binary/essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill


(870, 218)

In [27]:
# load concept code (cc) tagged essays
cc_tagged_tr, cc_tagged_test = load_tagged_essays(tag_predictions_folder)
len(cc_tagged_tr), len(cc_tagged_test)

/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM_fixed/essays_test_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill
/Users/simon.hughes/Google Drive/Phd/Data/SkinCancer/Thesis_Dataset/Predictions/Bi-LSTM_fixed/essays_train_bi_directional-True_hidden_size-256_merge_mode-sum_num_rnns-2_use_pretrained_embedding-True.dill


(870, 218)

## Load Raw Essays (Untagged)

In [28]:
# do I need to do this? good for validation below, but not needed otherwise
with open(training_pickled, "rb+") as f:
    untagged_essays_train = pickle.load(f)

test_config = get_config(test_folder)
untagged_essays_test = load_process_essays(**test_config)

len(untagged_essays_train), len(untagged_essays_test) # 902, 226

218 files found
218 essays processed


(870, 218)

## Validate all essay sets are equal and the same

In [29]:
def names_the_same(essay_sets):
    unique_fnames = [] # list of sets of str (fnames)
    for essay_collection in essay_sets:
        names = set()
        for e in essay_collection:
            names.add(e.name)
        unique_fnames.append(names)
    for a in unique_fnames:
        print(len(a))
        for b in unique_fnames:
            assert len(a) == len(b), "lens don't match"
            assert a == b, "don't match"

In [30]:
names_the_same([ana_tagged_tr, cc_tagged_tr, untagged_essays_train])

870
870
870


In [31]:
names_the_same([ana_tagged_test, cc_tagged_test, untagged_essays_test])

218
218
218


In [32]:
def essays_2_hash_map(essays):
    lu = {}
    for e in essays:
        lu[e.name] = e
    return lu

In [33]:
# checks the number of words and sentences are the same for 2 sets of essays
def validate_tagged_essays(essays_a, essays_b):
    # make sure obj is not the same
    assert essays_a != essays_b
    print("Validating", len(essays_a), "essays")
    assert len(essays_a) == len(essays_b), "Lens don't match"
    
    a_hmap = essays_2_hash_map(essays_a)
    b_hmap = essays_2_hash_map(essays_b)
    
    # same essays?
    assert a_hmap.keys() == b_hmap.keys()
    intersect = set(a_hmap.keys()).intersection(b_hmap.keys())
    assert len(intersect) == len(a_hmap.keys())
    assert len(a_hmap.keys()) > 1    
    assert len(a_hmap.keys()) == len(b_hmap.keys())
    
    for key, a_essay in a_hmap.items():
        b_essay = b_hmap[key]
        # assert NOT the same obj ref
        assert a_essay != b_essay
        assert len(a_essay.sentences) == len(b_essay.sentences)
        assert len(a_essay.sentences) > 0
        assert len(b_essay.sentences) > 0
        for i in range(len(a_essay.sentences)):
            a_sent = a_essay.sentences[i]
            b_sent = b_essay.sentences[i]
            # the same lists?
            assert a_sent == b_sent
            assert len(a_sent) == len(b_sent)
            if not len(a_sent) == len(b_sent):
                print(key, "\tsent-ix:", i, "lens", len(a_sent), len(b_sent))
            for wd_ix, (a_wd, a_tags) in enumerate(a_sent):
                b_wd, b_tags = b_sent[wd_ix]
                assert a_wd   == b_wd,   "Words don't match: {a} - {b}".format(a=a_wd, b=b_wd)
                assert a_tags == b_tags, "Tags don't match: {a} - {b}".format(a=str(a_tags), b=str(b_tags))
                
    print("Validation Passed")
    return None

validate_tagged_essays(untagged_essays_train, ana_tagged_tr)
validate_tagged_essays(cc_tagged_tr, ana_tagged_tr)
validate_tagged_essays(untagged_essays_test, ana_tagged_test)
validate_tagged_essays(cc_tagged_test, ana_tagged_test)

Validating 870 essays
Validation Passed
Validating 870 essays
Validation Passed
Validating 218 essays
Validation Passed
Validating 218 essays
Validation Passed


## Merge the Two Sets of Tagged Essays
* The anaphora essays were not tagged with concept codes, and vice versa, so need to merge

In [34]:
def merge_tagged_essays(tagged_ana, tagged_cc):
    ana_hmap = essays_2_hash_map(tagged_ana)
    cc_hmap = essays_2_hash_map(tagged_cc)
    
    assert ana_hmap.keys() == cc_hmap.keys()
    assert len(ana_hmap.keys()) == len(cc_hmap.keys())
    
    for key, cc_essay in cc_hmap.items():
        ana_essay = ana_hmap[key]
        assert len(ana_essay.sentences) == len(cc_essay.sentences)
        cc_essay.ana_tagged_sentences = ana_essay.pred_tagged_sentences
        for i in range(len(ana_essay.sentences)):
            ana_sent = ana_essay.sentences[i]
            ana_ptags = ana_essay.pred_tagged_sentences[i]
            cc_sent = ana_essay.sentences[i]
            cc_ptags = cc_essay.pred_tagged_sentences[i]
            assert len(ana_sent) == len(cc_sent)
            assert len(cc_sent) == len(cc_ptags)
    return tagged_cc

merged_essays_tr   = merge_tagged_essays(tagged_ana=ana_tagged_tr, tagged_cc=cc_tagged_tr)
merged_essays_test = merge_tagged_essays(tagged_ana=ana_tagged_test, tagged_cc=cc_tagged_test)
len(merged_essays_tr), len(merged_essays_test)

(870, 218)

## Re-Compute Accuracy Metrics

In [35]:
def norm_tally(tally):
    total = sum(tally.values())
    norm_tally = {}
    for tag, freq in tally.items():
        norm_tally[tag] = freq/total
    return norm_tally

### Get labels


In [36]:
tally = defaultdict(int)
for e in untagged_essays_train:
    for sent in e.sentences:
        for wd, tags in sent:
            for t in tags:
                tally[t] +=1

lst_all_tags = list(tally.keys())
regular_tags = sorted(set((t for t in lst_all_tags if t[0].isdigit())), key = lambda s: int(s.replace('b','')))
assert EMPTY_TAG not in regular_tags, "Empty tag in list of regular tags"
regular_tags

['1', '2', '3', '4', '5', '6', '11', '12', '50']

### Compare Label Distributions

In [37]:
cc_tally = dict([k,v] for k,v in tally.items() if k in set(regular_tags))
sorted(norm_tally(cc_tally).items(), key = lambda tpl: tpl[0])

[('1', 0.14999496661185865),
 ('11', 0.01171101640884534),
 ('12', 0.010838562464346835),
 ('2', 0.1533505587060837),
 ('3', 0.09439280561054998),
 ('4', 0.07409147344048858),
 ('5', 0.13341834166638702),
 ('50', 0.2954263279755713),
 ('6', 0.07677594711586859)]

In [38]:
ptag_tally = defaultdict(int)
for e in merged_essays_tr:   
    for sent in e.pred_tagged_sentences:
        for tag in sent:
            ptag_tally[tag] +=1
            
norm_ptag_tally = norm_tally(dict([(k,v) for k,v in ptag_tally.items() if k != EMPTY_TAG]))
sorted(norm_ptag_tally.items(), key = lambda tpl: tpl[0])

[('1', 0.15222876366694701),
 ('11', 0.00963694981777404),
 ('12', 0.009882253994953743),
 ('2', 0.1559784132324082),
 ('3', 0.09668488926268573),
 ('4', 0.0787426408746846),
 ('5', 0.13589851415755536),
 ('50', 0.2989206616204093),
 ('6', 0.062026913372582)]

In [39]:
list(ptag_tally.keys())

['Empty', '50', '3', '2', '12', '4', '5', '6', '1', '11']

### Get Predicted and Actual Labels

In [40]:
def get_label(tag, expected_tag_set):
    if tag in expected_tag_set:
        return 1
    else:
        return 0

def get_wd_level_lbs(essays, expected_tags):
    expected_tags = set(expected_tags)
    ysbycode = defaultdict(list)
    for e in essays:
        for sent in e.sentences:
            for wd, tag_set in sent:
                for etag in expected_tags:
                    ysbycode[etag].append(get_label(etag, tag_set))
    return ysbycode    

# for pred tags
def get_wd_level_preds(essays, expected_tags):
    expected_tags = set(expected_tags)
    ysbycode = defaultdict(list)
    for e in essays:
        for sentix in range(len(e.sentences)):
            p_ccodes = e.pred_tagged_sentences[sentix]
            p_ana_tags = e.ana_tagged_sentences[sentix]
            assert len(p_ccodes) == len(p_ana_tags), "Different length predictions"
            for wordix in range(len(p_ccodes)):
                ptag_set = set([p_ccodes[wordix], p_ana_tags[wordix]])
                assert len(ptag_set) >=1, "No tags found"
                for exp_tag in expected_tags:
                    ysbycode[exp_tag].append(get_label(exp_tag, ptag_set))    
    return ysbycode   

#### Validate Metrics on Concept Codes

In [41]:
#TODO - get predicted ccodes and anaphora labels, merge into one set of preds and filter by expected_tags.
# this give flexibility to look at anaphora, cc or both
act_cc_ys_bycode = get_wd_level_lbs(merged_essays_tr, regular_tags)
pred_cc_ys_bycode = get_wd_level_preds(merged_essays_tr, regular_tags)

assert len(act_cc_ys_bycode.keys()) == len(pred_cc_ys_bycode.keys()) == len(regular_tags), "Miss-matched codes"
assert len(act_cc_ys_bycode["50"]) == len(pred_cc_ys_bycode["50"]), "Different numbers of words"

print(len(act_cc_ys_bycode), len(act_cc_ys_bycode["50"]), len(pred_cc_ys_bycode), len(pred_cc_ys_bycode["50"]))

9 145471 9 145471


In [42]:
metrics = ResultsProcessor.compute_metrics(act_cc_ys_bycode, pred_cc_ys_bycode)
mean_metrics = ResultsProcessor.compute_mean_metrics(act_cc_ys_bycode, pred_cc_ys_bycode)
#metrics.update(mean_metrics)
metrics

{'1': Recall: 0.8036, Precision: 0.8269, F1: 0.8151, Accuracy: 0.9888, Codes:  4470,
 '11': Recall: 0.6676, Precision: 0.8473, F1: 0.7468, Accuracy: 0.9989, Codes:   349,
 '12': Recall: 0.6006, Precision: 0.6879, F1: 0.6413, Accuracy: 0.9985, Codes:   323,
 '2': Recall: 0.7807, Precision: 0.8016, F1: 0.7910, Accuracy: 0.9870, Codes:  4570,
 '3': Recall: 0.8009, Precision: 0.8166, F1: 0.8087, Accuracy: 0.9927, Codes:  2813,
 '4': Recall: 0.7844, Precision: 0.7708, F1: 0.7776, Accuracy: 0.9932, Codes:  2208,
 '5': Recall: 0.8441, Precision: 0.8654, F1: 0.8546, Accuracy: 0.9921, Codes:  3976,
 '50': Recall: 0.8545, Precision: 0.8819, F1: 0.8680, Accuracy: 0.9843, Codes:  8804,
 '6': Recall: 0.6726, Precision: 0.8695, F1: 0.7585, Accuracy: 0.9933, Codes:  2288}

In [43]:
mean_metrics[__MICRO_F1__]

{'accuracy': 0.9920892976759782,
 'data_points': 1309239.0,
 'f1_score': 0.822462588065893,
 'num_codes': 29801.0,
 'precision': 0.8406924586487244,
 'recall': 0.8050065434045838}

#### Validate Metrics on Anaphora Tags

In [44]:
#TODO - get predicted ccodes and anaphora labels, merge into one set of preds and filter by expected_tags.
# this give flexibility to look at anaphora, cc or both
ana_tags = [ANAPHORA]

act_ana_ys_bycode  = get_wd_level_lbs(merged_essays_tr, ana_tags)
pred_ana_ys_bycode = get_wd_level_preds(merged_essays_tr, ana_tags)

assert len(act_ana_ys_bycode.keys()) == len(pred_ana_ys_bycode.keys()) == len(ana_tags), "Miss-matched codes"
assert len(act_ana_ys_bycode[ANAPHORA]) == len(pred_ana_ys_bycode[ANAPHORA]), "Different numbers of words"

print(len(act_ana_ys_bycode), len(act_ana_ys_bycode[ANAPHORA]), len(pred_ana_ys_bycode), len(pred_ana_ys_bycode[ANAPHORA]))

1 145471 1 145471


In [45]:
metrics_ana = ResultsProcessor.compute_metrics(act_ana_ys_bycode, pred_ana_ys_bycode)
mean_metrics_ana = ResultsProcessor.compute_mean_metrics(act_ana_ys_bycode, pred_ana_ys_bycode)
#metrics.update(mean_metrics)
print(metrics_ana)

{'Anaphor': Recall: 0.3200, Precision: 0.4565, F1: 0.3762, Accuracy: 0.9965, Codes:   475}


In [46]:
mean_metrics_ana[__MICRO_F1__]

{'accuracy': 0.9965353919337875,
 'data_points': 145471.0,
 'f1_score': 0.37623762376237624,
 'num_codes': 475.0,
 'precision': 0.45645645645645644,
 'recall': 0.32}